In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
PSQL_USER=os.environ["PSQL_USER"]
PSQL_PASSWORD=os.environ["PSQL_PASSWORD"]
MONGODB_URI=os.environ["MONGODB_URI"]
MONGODB_DATABASE=os.environ['MONGODB_DATABASE']
MONGODB_COLLECTION=os.environ['MONGODB_COLLECTION']

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType , StringType
from pyspark.sql.functions import *

In [4]:
# create a local SparkSession
spark = SparkSession.builder\
        .appName("PYSPARK - PostgreSQL to MongoDB")\
        .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector:10.0.5,org.postgresql:postgresql:42.6.0")\
        .getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true


:: loading settings :: url = jar:file:/home/kali/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kali/.ivy2/cache
The jars for the packages stored in: /home/kali/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5385167a-c554-49b5-89fc-db9136532541;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 6734ms :: artifacts dl 15ms
	:: modules in use:
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongo

23/04/11 05:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
listing = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:database1") \
    .option("dbtable", "listing") \
    .option("user", PSQL_USER) \
    .option("password", PSQL_PASSWORD)\
    .option("driver","org.postgresql.Driver")\
    .load()

In [6]:
host=spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:database1") \
    .option("dbtable", "host") \
    .option("user", PSQL_USER) \
    .option("password", PSQL_PASSWORD) \
    .option("driver","org.postgresql.Driver")\
    .load()

In [7]:
data=listing.join(host, listing.host_id==host.host_id).drop(host.host_id)

In [8]:
load=data.write\
        .format("mongodb")\
        .option("checkpointLocation", "/tmp/pyspark/")\
        .option("forceDeleteTempCheckpointLocation", "true")\
        .option("connection.uri", MONGODB_URI )\
        .option("database", MONGODB_DATABASE )\
        .option("collection", MONGODB_COLLECTION)\
        .mode("append")\
        .save()

23/04/11 05:25:55 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/04/11 05:25:55 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [9]:
spark.stop()